In [ ]:
!pip install -q --upgrade google-generativeai langchain-google-genai chromadb pypdf
!sudo apt -y -qq install tesseract-ocr libtesseract-dev
!sudo apt-get -y -qq install poppler-utils libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig
!pip install langchain
!pip install -U langchain-community
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.3/717.3 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
from IPython.display import display
from IPython.display import Markdown
import textwrap
import google.generativeai as genai
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
warnings.filterwarnings("ignore")
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.embeddings import GooglePalmEmbeddings
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
class HFACS:

    def __init__(self, chunk_size, chunk_overlap, k):
        self.GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
        genai.configure(api_key=self.GOOGLE_API_KEY)
        self.GeminiProModel = genai.GenerativeModel(model_name="gemini-pro")
        self.langchainModel = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=self.GOOGLE_API_KEY)
        self.context = """
The Human Factors Analysis and Classification System (HFACS) framework categorizes human error into four levels:

1. **Unsafe Acts**: These Influences or Factors are closely tied to the mishap and described as active failures or actions committed by an individual that result in a human event.
These are divided into 4 categories:
(i) *Decision error*: Actions and plans unintentionally chosen by operators are inappropriate and lead to unsafe situations.
(ii) *Skill-based error*: Unintentional errors that can be reduced through learning. The actions are related to a routine task or procedure.
(iii) *Perceptual error*: Misperception of an object, equipment, environment, threat, or situation; visual, auditory, proprioceptive, or vestibular illusions; cognitive or attention failures.
(iv) *Routine violation*: Intentionally ignoring established rules and procedures.

2. **Preconditions for Unsafe Acts**: When environmental factors or conditions of individuals affect performance and this condition contributes to an error.
These are divided into 9 categories:
(i) *Physical environment*: When factors such as weather, climate, fog, brownout (dust or sand storm) or whiteout (snow storm) affect the actions of individual.
(ii) *Technical environment*: These are factors when automation or the design of the workspace affects the actions of an individual.
(iii) *Personnel Factors*: When teamwork or self-imposed stress affect practices, conditions, or actions of an individual’s performance. Communication Refers to the interactions among individuals, crews, and teams involved with the preparation and/or performance of a mission that resulted in an error or an unsafe situation.
(iv) *Self-Imposed Stress*: These are factors when an individual demonstrates disregard for rules that govern the individual’s readiness to perform or exhibits poor judgment when it comes to readiness and this contributes to an error.
(v) *Individual Factors*: This is a factor in a mishap if cognitive, psycho-behavioral, adverse physical state, or physical/mental limitations affect practices, conditions or actions of individuals and result in human error or an unsafe situation.
(vi) *Awareness Cognitive Factors*: These are factors when attention management conditions affect an individual’s performance and this state contributes to an error.
(vii) *Adverse Psychological*: These are factors when an individual experiences a psychological event that compromises performance and this condition contributes to an error.
(viii) *Medical / Mental*: These are factors when an individual, temporarily or permanently, lacks the physical or mental capabilities to cope with a situation and this condition contributes to an error.
(ix) *Perception*: These are factors resulting in degraded sensory inputs (visual, auditory or vestibular) that create a misperception of an object, threat or situation.

3. **Unsafe Supervision**: Is a factor in a mishap if the methods, decisions or policies of the supervisory chain of command directly affect practices, conditions, or individual actions and result in human error or an unsafe situation.
These are divided into 4 categories:
(i) *Inadequate supervision*: This is a factor in a mishap when supervision proves inappropriate or improper and fails to identify hazard, recognize and control risk, provide guidance, training and/or oversight and results in human error or an unsafe situation.
(ii) *Planned inappropriate operation*: These are factors when supervision fails to adequately assess the hazards associated with an operation and allows for unnecessary risk. It is also a factor when supervision allows non-proficient or inexperienced personnel to attempt missions beyond their capability or when crew or flight makeup is inappropriate for the task or mission.
(iii) *Failed to correct problem*: These are factors in a mishap when supervision fails to correct known deficiencies in documents, processes or procedures, or fails to correct inappropriate or unsafe actions of individuals, and this lack of supervisory action creates an unsafe situation.
(iv) *Supervisory violation*: These are factors in a mishap when supervision while managing organizational assets willfully disregards instructions, guidance, rules, or operating instructions and this lack of supervisory responsibility creates an unsafe situation.

4. **Organizational Influences**: This level describes the methods, decisions or policies of an organization and how they affect both supervisory and individual task and mission accomplishment.
These are divided into 3 categories:
(i) *Resource management*: These are factors when the people or products an organization needs to accomplish a mission are inadequate or improper and this condition reduces system safety or increases operational risk.
(ii) *Organizational process*: These are factors when organizational processes, procedures, risk management and oversight negatively impact operations.
(iii) *Organizational Culture / Climate*: These are factors when the attitudes, values, beliefs or morale negatively impact operations and/or increase operational risk.
"""
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.kVal = k

    def toMarkdown(self, text):
        text = text.replace('•', '  *')
        return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

    def geminiResponse(self, prompt):
        response = self.GeminiProModel.generate_content(prompt)
        return response.text

    def langChainResponse(self, prompt):
        response = self.langchainModel.invoke(prompt)
        return response.content

    def RAGResponse(self, prompt):
        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
        genai.configure(api_key=GOOGLE_API_KEY)
        RAGModel = ChatGoogleGenerativeAI(model="gemini-pro",
                                               google_api_key=GOOGLE_API_KEY,
                                               temperature=0.1,
                                               convert_system_message_to_human=True)
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap)
        texts = text_splitter.split_text(self.context)
        embeddings = GooglePalmEmbeddings(model="models/embedding-001", google_api_key=self.GOOGLE_API_KEY)
        vector_index = FAISS.from_texts(texts, embeddings).as_retriever(search_kwargs={"k": self.kVal})
        template = """Use the following pieces of context you have to classify the following incident report according to the HFACS categories. You will be given summary of the incident happened. Apart from that, you will also be given with the safety issue observed by the officers. Your job is to output the level and the category to which the incident belongs to. Also, do give the reason why you are choosing that particular category. Make sure that reason should be understandable for a user. Do recheck once if needed. Give me the best possible answer for the given safety issue. Make sure you give importance to safety issue comparitively than summary. Make sure that you output level, category and reason for the given prompt. Do recheck yourself so that the category you output falls in the level we have. Keep the answer as concise as possible.
        {context}
        Question: {question}
        Helpful Answer:"""
        QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
        qa_chain = RetrievalQA.from_chain_type(
            RAGModel,
            retriever=vector_index,
            return_source_documents=True,
            chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
        )
        response = qa_chain({"query": prompt})
        return response["result"]

    # def preProcess(self, response):
    #     level = None
    #     category = None
    #     reason = None

    #     if "Level:" in response and "Category" in response and "Reason:" in response:
    #         nanocode_start = response.index("Nanocode:") + len("Nanocode: ")
    #         reason_start = response.index("Reason:")
    #         nanocode = response[nanocode_start:reason_start].strip()

    #         reason_start += len("Reason: ")
    #         reason = response[reason_start:].strip()

    #     return nanocode, reason

    # def preProcess1(self, issues):
    #     sentences = re.split(r'\n{1,2}', issues)
    #     return sentences

    # def ReportClassifier(self, summary, issues):
    #     safety_issues = self.preProcess1(issues)
    #     n = len(safety_issues)
    #     for issue in safety_issues:
    #       sum_and_issue = "Summary :" + summary + "Safety Issue:" + issue
    #       response = self.RAGResponse(sum_and_issue)
    #       nanocode, reason = self.preProcess(response)
    #       print("Nanocode for safety issue is ", nanocode )
    #       print("Reason for choosing it is ", reason)
    #       print("\n")

In [ ]:
obj = HFACS(800, 100, 3)

In [ ]:
text = obj.RAGResponse("Summary : On 7 October 2022, a deckhand from the UK registered gill netter Eder Sands fell overboard while the vessel was shooting a net approximately 150 nautical miles west of Ireland. Despite an extensive 21-hour search that involved other vessels and fixed-wing aircraft, the crew member was not found. Safety Issue: The lifejackets used by the crew were compliant but unsuitable for the vessel’s operation because they were not fitted with a light, personal locator beacon, crotch strap or spray hood.")

In [ ]:
obj.toMarkdown(text)

> **Level:** Organizational
> **Category:** Inadequate supervision
> **Reason:** The safety issue observed was that the lifejackets used by the crew were compliant but unsuitable for the vessel's operation because they were not fitted with a light, personal locator beacon, crotch strap or spray hood. This indicates that the supervision failed to adequately assess the hazards associated with the operation and allowed for unnecessary risk.

In [ ]:
text = obj.RAGResponse("Summary : On 2 October 2022, the mono-hulled offshore powerboat Awesome was returning to its home port when it unexpectedly veered to starboard, then rolled violently to port, ejecting 10 occupants. Awesome ended up in a vertical position, with its bow in the air, briefly staying afloat before it started to sink by the stern. Two of the occupants did not survive the accident. Safety Issue: Awesome had seating for five people; at the time of the accident there were 11 people on board, including four children.")

In [ ]:
obj.toMarkdown(text)

> **Level:** Organizational
> **Category:** Inadequate supervision
> **Reason:** The safety issue observed was that Awesome had seating for five people, but at the time of the accident there were 11 people on board, including four children. This indicates that supervision failed to adequately assess the hazards associated with the operation and allowed for unnecessary risk.

In [ ]:
text = obj.RAGResponse("Summary : At about 0300 on 18 February 2021, a deckhand fell overboard from the twin rig stern trawler Copious (LK 985) approximately 30 nautical miles south-east of the Shetland Islands. The deckhand was conscious, wearing a lifejacket and was quickly brought alongside the vessel. However, the crew’s attempts to recover the casualty back on board were unsuccessful. He was unresponsive when recovered from the water by a coastguard helicopter and pronounced dead on arrival at hospital. Safety Issue: When he lost consciousness in the water, the incorrectly worn lifejacket did not hold his airways clear and he drowned due to complications of immersion.")

In [ ]:
obj.toMarkdown(text)

> **Level:** Organizational
> **Category:** Unsafe Supervision
> **Reason:** The safety issue observed was that the lifejacket was incorrectly worn, which led to the deckhand drowning. This suggests that there was a failure in supervision to ensure that the deckhand was properly trained and equipped for the task.

In [ ]:
text = obj.RAGResponse("Summary : On the morning of 12 January 2022, two men were found unconscious in the covered cockpit of the privately owned sports cruiser Emma Louise, which was moored in Port Hamble Marina. The men had left the boat’s petrol engine running during the previous evening, likely to maintain power to some of the boat’s systems. Exhaust gas had built up in the covered cockpit area, causing them to pass out and die due to carbon monoxide (CO) poisoning. Safety Issue: With no CO alarm fitted on board the men were not alerted to the danger of CO poisoning before they succumbed.")

In [ ]:
obj.toMarkdown(text)

> **Level:** Preconditions for Unsafe Acts
> **Category:** Physical Environment
> **Reason:** The physical environment, specifically the enclosed cockpit area, allowed exhaust gas to build up, leading to carbon monoxide poisoning. The absence of a CO alarm contributed to the unsafe situation by not alerting the men to the danger.

In [ ]:
text = obj.RAGResponse("Summary : On April 23, 2023, about 2242 local time, the towing vessel Susan K was pushing 25 barges downbound on the Lower Mississippi River when the tow struck the center bridge pier on the Natchez-Vidalia Bridge, which connects the cities of Natchez, Mississippi, and Vidalia, Louisiana. One barge sank, and two other barges were damaged; the Susan K was undamaged. No pollution or injuries were reported. Damage to the barges and cargo was estimated at $2 million. Safety Issue : ​We determined that the probable cause of the contact of the Susan K tow with the Natchez-Vidalia Bridge was the captain’s complacency, which resulted in his inattention to the tow’s position as it approached the bridge.​​")

In [ ]:
obj.toMarkdown(text)

> **Level:** Supervisory
> **Category:** Inadequate supervision
> **Reason:** The captain's complacency, which resulted in his inattention to the tow's position as it approached the bridge, is an example of inadequate supervision. The captain failed to identify the hazard of the tow's position and to take appropriate action to avoid the bridge. This lack of supervision resulted in the tow striking the bridge and causing damage to the barges and cargo.

In [ ]:
obj = HFACS(1500, 150, 10)

In [ ]:
# issue = "Summary : ​On August 11, 2022, about 1834 local time, the containership Maunalei was transiting the North Pacific Ocean, about 245 miles northwest of the entrance to the Columbia River, en route to Portland, Oregon, when the crew intentionally shut down the main engine due to problems in the controllable pitch propeller system, resulting in a loss of propulsion. The vessel’s controllable pitch propeller system may have lost up to 1,632 gallons of hydraulic oil. There were no injuries reported. Damage to the vessel was estimated at $3.03 million. Safety Issue : ​We determined that the probable cause of the loss of propulsion on the containership Maunalei was a crack developing in a controllable pitch propeller blade base and progressing into a fracture due to high cycle fatigue as a result of the blade not meeting manufacturer design specifications.​"
# issue = "Summary : On November 25, 2022, about 0030 local time, the commercial fishing vessel Captain Alex was fishing in the Gulf of Mexico about 15 nautical miles south southwest of Galveston, Texas, when the vessel began flooding. The four crewmembers on board were unable to stem the flooding, and they abandoned the vessel to a responding US Coast Guard boat. The Captain Alex later sank, and an oil sheen and debris field were visible; a reported 17,000 gallons of diesel fuel were on board. There were no injuries. The Captain Alex, with a value of $500,000, was a total loss. Safety Issue : ​​We determined that the probable cause of the sinking of the fishing vessel Captain Alex was uncontrolled flooding through a hole—possibly caused by steel hull plating deterioration—beneath the engine room.​​​​​"
issue = "Summary : ​On November 20, 2022, at 1256 local time, the liftboat L/B Robert was reported to be listing (tilting) alongside a stationary oil platform in the Gulf of Mexico near South Marsh Island Block 137, about 80 miles southeast of Lake Charles, Louisiana. The crew had evacuated 2 days prior due to forecasted adverse weather in the area, leaving the vessel elevated out of the water and unattended. Following the casualty, vessel crewmembers, salvors, and personnel from the operating company assessed the damage and facilitated the tow of the vessel into port. No pollution or injuries were reported. Damage to the vessel and cargo was estimated at $6.9 million.​ Safety Issue : ​We determined that the probable cause of the tilting of the elevated liftboat L/B Robert was gale-force winds combined with waves that exceeded the vessel’s air gap, causing vessel movement, which led to a leg sliding into an adjacent can hole in the seafloor.​"

In [ ]:
text = obj.RAGResponse(issue)

In [ ]:
obj.toMarkdown(text)

> **Level:** Preconditions for Unsafe Acts
> **Category:** Physical Environment
> **Reason:** The safety issue mentions that "gale-force winds combined with waves that exceeded the vessel's air gap" caused the vessel to tilt. These environmental factors affected the actions of the vessel and contributed to the incident.